# 01.04 rootfinding: newtons method

aka newton-[raphson](https://en.wikipedia.org/wiki/Joseph_Raphson). newtons is a variant of FPI.

to find root of $f(x) = 0$, start with guess $x_0$. draw tangent line at $f(x_0)$. ie, where $f'(x_0)$ intersects $x$-axis is the next iteration $x_1$. ie,
</br></br>

$$tan\theta = \frac{f(x_0)}{x_0-x_1} = f'(x_0) \Rightarrow x_1 = x_0 - \frac{f(x_0)}{f'(x_0)}.$$

###### code, visual: newtons

given $f(x) = -\frac{1}{2}x^2 + x + 1.5$.

In [11]:
# requires prior execution of fpi_expanded()
# repurposed bisection animation for newtons method

if __name__ == "__main__":

  import math as m
  import numpy as np
  import scipy as sp
  import statistics as st
  from tabulate import tabulate

  if False: # original set of points # for completeness
    xs = np.array([-1,-0.5,0])
    ys = np.array([0,0.875,1.5])
    pc = np.polyfit(xs,ys,2)

    xs_plot = np.linspace(xs.min()-0.5,xs.max()+0.5,101)
    ys_plot = np.polyval(pc,xs_plot)

  # prob-def
  f = lambda x: -pow(x,2)/2 + x + 1.5
  df = lambda x: 1 - x
  g = lambda x: x - f(x)/df(x)

  # prod-def, interval
  ab = (-1,0)
  h = 0.1
  h_plot = h/10

  # prob-def, generalized
  s_title = "some other example for newtons: $-\\frac{1}{2}x^2 + x + 1.5$"
  s_label = "{-\frac{1}{2}x^2 + x + 1.5}{1-x}"
  s_label = f"$g(x) = x - \\frac{s_label}$"

  # prob-def, run-time
  x0 = st.mean(ab)
  root = (sp.optimize.root(f,[x0])).x
  ws = fpi_expanded(g,x0,worksheet=True)

  # cfg, runtime
  iterations = 3

  # exxing types
  we = []; e0 = 0
  for w in ws:
    ei = abs(w[1]-root)
    if e0 != 0:
      ed = ei/pow(e0,2)
    else:
      ed = None
    e0 = ei
    we.append([w[0],w[1],ei,ed])

  # output, table
  if False:
    s_ifmt = "03d"
    if np.max(np.absolute(ws)[:,1]) >= 10:
      s_ffmt = ".8e"
    else:
      s_ffmt = ".8f"
    print(f"\n{s_title}. x0 = {x0}.\n")
    if len(we) > 10:
      print(tabulate(we[0:6][:],headers=["i","x[i]","e[i]",'"/e[i-1]^2'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
      print(tabulate(we[-5:][:],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
    else:
      print(tabulate(we,headers=["i","x[i]","e[i]",'"/e[i-1]^2'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))

  # output, cobweb plot
  if False:
    import matplotlib.pyplot as plt

    # g(x)
    s_grey = "0.5"
    plt.plot([ab[0]-h,ab[1]+h],[ab[0]-h,ab[1]+h],ls=":",c=s_grey)
    xs_plot = np.arange(ab[0]-h,ab[1]+h_plot/2,h_plot)
    gs_plot = g(xs_plot)
    plt.plot(xs_plot,gs_plot,label=s_label,c=s_grey)

    # fpi
    s_blue = "C00"
    xs = []; ys = []
    imax = min(len(ws),5)
    for i in range(1,imax):
      xs.append(ws[i-1][1])
      ys.append(ws[i][1])
    plt.scatter(xs,ys,c=s_blue,marker="*")

    # fpi vs x=y diagonal
    y0 =0
    for x,y in zip(xs,ys):
      if y0 == 0:
        plt.plot([x,x],[y0,y],label="fpi",c=s_blue,alpha=0.5)
      else:
        plt.plot([x,x],[y0,y],c=s_blue,alpha=0.5)
      plt.arrow(x,y,y-x,0,color=s_blue,alpha=0.5,length_includes_head=True,head_width=0.025)
      y0 = y

    plt.title(f"{s_title}, {s_label}")
    plt.grid()
    plt.legend()
    plt.show()

  # output, animiation # True requires "ani" in a separate code cell
  if True:
    from itertools import count
    import matplotlib.animation
    import matplotlib.pyplot as plt
    #import numpy as np # already

    # data, true, scatter
    h = 0.1
    xs = np.arange(ab[0],ab[1]+h,h)
    ys = f(xs)

    # data, true, plot
    h_plot = h/10
    xs_plot_a = ab[0]-h
    xs_plot_b = ab[1]+h
    xs_plot = np.arange(xs_plot_a,xs_plot_b+h_plot,h_plot)
    ys_plot = f(xs_plot)

    # plot, animation
    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 100
    plt.ioff()
    fig,ax = plt.subplots()
    index = count() # used in animate()

    # plot, actual
    ax.plot(xs_plot,ys_plot,zorder=1)
    ax.set_xlim(xs_plot_a,xs_plot_b)
    ymin = m.floor(ys_plot.min()/h)*h - h
    ymax = m.ceil(ys_plot.max()/h)*h + h
    ax.set_ylim(ymin,ymax)
    ax2 = ax.twinx()
    ax.set_title(s_title)
    ax.grid()

    # plot, iterations
    zorder = 10
    ws = np.array(ws) # for referential
    def animate(t):
      ax2.cla()
      k = next(index)
      if k == 2:
        ax.scatter(root,f(root),color="C01",marker="*",s=200,zorder=2)
      if k > 2:
        step = 3
        imax = k - 2 # first frame on ax2
        for iset in range(0,imax,step):
          i = iset // step
          color = f"C{i+2:02d}" # C02,C03,C04,...

          jmax = min(imax-iset,step)
          for j in range(jmax):
            if j > 1:
              x = ws[i,1]
              x1 = ws[i+1,1]
              y = f(x)
              ya = y*((xs_plot_a-x)/(x-x1) + 1)
              yb = y*((xs_plot_b-x)/(x-x1) + 1)
              ax2.plot([xs_plot_a,xs_plot_b],[ya,yb],label="$m_{i}$",color=color)
            else:
              if j == 0:
                x = ws[i,1]
                xs_j = [x]
                ys_j = [0]
                acbs = ["x"]
              elif j == 1:
                x = ws[i,1]
                xs_j = [x]
                ys_j = [f(x)]
                acbs = ["y"]
              ax2.scatter(xs_j,ys_j,color=color,zorder=zorder+i)
              for acb,x,y in zip(acbs,xs_j,ys_j):
                label = "$" + acb+ "_{" + str(i) + "}$"
                ax2.text(x-h_plot*2,y-h,label,color=color,fontweight="bold",zorder=zorder*2)

      # inside ani function!!
      ax2.set_xlim(xs_plot_a,xs_plot_b)
      ax2.set_ylim(ymin,ymax)

    ani = matplotlib.animation.FuncAnimation(fig,animate,frames=11,interval=11)

    # show static
    if False:
      plt.show()


In [12]:
ani

##### algorithm

<b>newtons method</b>
</br></br>

\begin{align}
  x_{i+1} &= x_i - \frac{f(x_i)}{f'(x_i)}, \quad i = 0,1,2,3,\dots \\
  \\
  &= g(x_i), \quad g(x) = x - \frac{f(x)}{f'(x)}.
\end{align}
</br>

```
function g(x)
  return x - f(x)/df(x)

function fpi(g,x0,epsilon,imax)

```

##### example 11

solve $f(x) = x^3 + x - 1$ using newtons.

\begin{align}
  f'(x) &= 3x^2 + 1 \\
  \\
  \Rightarrow x_{i+1} &= x_i - \frac{x_i^3 + x_i - 1}{3x_i^2 + 1} \\
  \\
  &= \frac{2x_i^3 + 1}{3x_i^2 + 1}, \quad\text{which looks strangely familiar...} \\
  \\
  &\Downarrow\quad x_0 = -0.7 \\
  \\
  x_1 &= \tfrac{2(-0.7)^3 + 1}{3(-0.7)^2 + 1} \approx 0.1271 \\
  x_2 &= \tfrac{2x_1^3 + 1}{3x_i^2+1} \approx 0.9577.
\end{align}

</br></br></br></br>

###### code, example 11

In [ ]:
# example 11 revisits example 01 # repurposed previous code

if __name__ == "__main__":

  import numpy as np
  import scipy as sp
  from tabulate import tabulate

  # prob-def
  f = lambda x : pow(x,3) + x - 1
  df = lambda x : 3*pow(x,2) + 1
  g4 = lambda x : x - f(x)/df(x) # newtons method

  ab = (-1,1)
  h = 0.1
  h_plot = h/10

  # prob-def, generalized
  g = g4
  s_title = f"example 11 ~ example 01 w newtons"
  s_label = "{x^3 + x - 1}{3x^2 + 1}"
  s_label = f"$g(x) = x - \\frac{s_label}$"

  # prob-def, run-time
  x0 = -0.7
  root = (sp.optimize.root(f,[x0])).x
  ws = fpi_expanded(g,x0,worksheet=True)

  # cfg, runtime
  iterations = 3

  # exxing types
  we = []; e0 = 0
  for w in ws:
    ei = abs(w[1]-root)
    if e0 != 0:
      ed = ei/pow(e0,2)
    else:
      ed = None
    e0 = ei
    we.append([w[0],w[1],ei,ed])

  # output, table
  if True:
    s_ifmt = "03d"
    if np.max(np.absolute(ws)[:,1]) >= 10:
      s_ffmt = ".8e"
    else:
      s_ffmt = ".8f"
    print(f"\n{s_title}. x0 = {x0}.\n")
    if len(we) > 10:
      print(tabulate(we[0:6][:],headers=["i","x[i]","e[i]",'"/e[i-1]^2'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
      print(tabulate(we[-5:][:],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
    else:
      print(tabulate(we,headers=["i","x[i]","e[i]",'"/e[i-1]^2'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))

  # output, cobweb plot
  if False:
    import matplotlib.pyplot as plt

    # g(x)
    s_grey = "0.5"
    plt.plot([ab[0]-h,ab[1]+h],[ab[0]-h,ab[1]+h],ls=":",c=s_grey)
    xs_plot = np.arange(ab[0]-h,ab[1]+h_plot/2,h_plot)
    gs_plot = g(xs_plot)
    plt.plot(xs_plot,gs_plot,label=s_label,c=s_grey)

    # fpi
    s_blue = "C00"
    xs = []; ys = []
    imax = min(len(ws),5)
    for i in range(1,imax):
      xs.append(ws[i-1][1])
      ys.append(ws[i][1])
    plt.scatter(xs,ys,c=s_blue,marker="*")

    # fpi vs x=y diagonal
    y0 =0
    for x,y in zip(xs,ys):
      if y0 == 0:
        plt.plot([x,x],[y0,y],label="fpi",c=s_blue,alpha=0.5)
      else:
        plt.plot([x,x],[y0,y],c=s_blue,alpha=0.5)
      plt.arrow(x,y,y-x,0,color=s_blue,alpha=0.5,length_includes_head=True,head_width=0.025)
      y0 = y

    plt.title(f"{s_title}, {s_label}")
    plt.grid()
    plt.legend()
    plt.show()

  # output, animiation # True requires "ani" in a separate code cell
  if True:
    from itertools import count
    import matplotlib.animation
    import matplotlib.pyplot as plt
    #import numpy as np # already

    # data, true, scatter
    h = 0.1
    xs = np.arange(ab[0],ab[1]+h,h)
    ys = f(xs)

    # data, true, plot
    h_plot = h/10
    xs_plot_a = ab[0]-h
    xs_plot_b = ab[1]+h
    xs_plot = np.arange(xs_plot_a,xs_plot_b+h_plot,h_plot)
    ys_plot = f(xs_plot)

    # plot, animation
    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 100
    plt.ioff()
    fig,ax = plt.subplots()
    index = count() # used in animate()

    # plot, actual
    ax.plot(xs_plot,ys_plot,zorder=1)
    ax.set_xlim(xs_plot_a,xs_plot_b)
    ax.set_ylim(f(xs_plot_a),f(xs_plot_b))
    ax2 = ax.twinx()
    ax.set_title(s_title)
    ax.grid()

    # plot, iterations
    zorder = 10
    ws = np.array(ws) # for referential
    def animate(t):
      ax2.cla()
      k = next(index)
      if k == 2:
        ax.scatter(root,f(root),color="C01",marker="*",s=200,zorder=2)
      if k > 2:
        step = 3
        imax = k - 2 # first frame on ax2
        for iset in range(0,imax,step):
          i = iset // step
          color = f"C{i+2:02d}" # C02,C03,C04,...

          jmax = min(imax-iset,step)
          for j in range(jmax):
            if j > 1:
              x = ws[i,1]
              x1 = ws[i+1,1]
              y = f(x)
              ya = y*((xs_plot_a-x)/(x-x1) + 1)
              yb = y*((xs_plot_b-x)/(x-x1) + 1)
              ax2.plot([xs_plot_a,xs_plot_b],[ya,yb],label="$m_{i}$",color=color)
            else:
              if j == 0:
                x = ws[i,1]
                xs_j = [x]
                ys_j = [0]
                acbs = ["x"]
              elif j == 1:
                x = ws[i,1]
                xs_j = [x]
                ys_j = [f(x)]
                acbs = ["y"]
              ax2.scatter(xs_j,ys_j,color=color,zorder=zorder+i)
              for acb,x,y in zip(acbs,xs_j,ys_j):
                label = "$" + acb+ "_{" + str(i) + "}$"
                ax2.text(x-h_plot*2,y-h*5/2,label,color=color,fontweight="bold",zorder=zorder*2)

      # inside ani function!!
      ax2.set_xlim(xs_plot_a,xs_plot_b)
      ax2.set_ylim(f(xs_plot_a),f(xs_plot_b))

    ani = matplotlib.animation.FuncAnimation(fig,animate,frames=11,interval=11)

    # show static
    if False:
      plt.show()



example 11 ~ example 01 w newtons. x0 = -0.7.

|   i |        x[i] |       e[i] |   "/e[i-1]^2 |
|-----|-------------|------------|--------------|
| 000 | -0.70000000 | 1.38232780 |              |
| 001 |  0.12712551 | 0.55520230 |   0.29055555 |
| 002 |  0.95767812 | 0.27535032 |   0.89327066 |
| 003 |  0.73482779 | 0.05249999 |   0.69244945 |
| 004 |  0.68459177 | 0.00226397 |   0.82139415 |
| 005 |  0.68233217 | 0.00000437 |   0.85266556 |
| 006 |  0.68232780 | 0.00000000 |   0.85407850 |


In [ ]:
ani

</br></br>

###### code, example 11, briefly


In [ ]:
if __name__ == "__main__":

  f = lambda x : pow(x,3) + x - 1
  df = lambda x : 3*pow(x,2) + 1

  g = lambda x : x - f(x)/df(x) # newtons method

  ws = fpi_expanded(g,x=-0.7,tol=1e-4,worksheet=True)
  iterations,root = ws[len(ws)-1]
  print(f"root {root} at {iterations} iterations.")


root 0.6823321742044841 at 5 iterations.


## 1 quadratic convergence

##### definition 10

let $e_i$ denote error after step $i$ of iterative method. iteration is <b>quadratically convergent</b> if
</br></br>

$$M = \lim_{i\rightarrow\infty}\frac{e_{i+1}}{e_i^2}<\infty.$$

##### theorem 11

let $f$ be twice continuously differentiable and $f(r) = 0$. if $f'(r) \ne 0$, then newtons is locally and quadratically convergent to $r$. error $e_i$ at step $i$ satisfies
</br></br>

$$\lim_{i\rightarrow\infty}\frac{e_{i+1}}{e_i^2} = M, \quad\text{where } M = \frac{f''(r)}{2f'(r)}.$$

###### proof


1. local convergence

note that newtons method is a particular form of FPI where
</br></br>

\begin{align}
  g(x) &= x - \frac{f(x)}{f'(x)}, \\ \\
  g'(x) &= 1 - \frac{f'(x)^2 - f(x)f''(x)}{f'(x)^2} = \frac{f(x)f''(x)}{f'(x)^2}.
\end{align}
</br>

$g'(r) = 0$ so locally convergent by theorem 06. $\checkmark$

2. quadratic convergence

derive newtons method with taylors formula. at $i$ steps,
</br></br>

\begin{align}
  f(r) &= f(x_i) + (r-x_i)f'(x_i) + \frac{(r-x_i)^2}{2}f''(c_i) \\
  \\
  &\quad\Downarrow \quad c_i\text{ between }x_i,r \\
  \\
  0 &= f(x_i) + (r-x_i)f'(x_i) + \frac{(r-x_i)^2}{2}f''(c_i) \\
  \\
  &\quad\Downarrow \\
  \\
  -\frac{f(x_i)}{f'(x_i)} &= r - x_i + \frac{(r-x_i)^2}{2}\frac{f''(c_i)}{f'(x_i)}, \quad f'(x_i) \ne 0 \\
  \\
  &\quad\Downarrow \\
  \\
  x_i - \frac{f(x_i)}{f'(x_i)} - r &= \frac{(r-x_i)^2}{2}\frac{f''(c_i)}{f'(x_i)} \\
  \\
  &\quad\Downarrow \quad e_i = |x_i-r| \\
  \\
  x_{i+1} - r &= e_i^2\frac{f''(c_i)}{f'(x_i)} \\
  \\
  &\quad\Downarrow \\
  \\
  e_{i+1} &= e_i^2\frac{f''(c_i)}{f'(x_i)}
\end{align}
</br>

bc $c_i$ is between $x_i,r$, it converges to $r$ as $x_i$ converges to $r$ and
</br></br>

$$\lim_{i\rightarrow\infty} \frac{e_{i+1}}{e_i^2} = \left| \frac{f''(r)}{2f'(r)} \right|,$$
</br>

which is the definition of quadratic convergence. $\checkmark \enspace \blacksquare$

ie,
</br></br>

$$e_{i+1} \approx Me_i^2, \quad M = \left| \tfrac{f''(r)}{2f'(r)} \right|, \enspace f'(r) \ne 0.$$
</br>

compare with linearly convergent methods with $e_{i+1} \approx Se_i$, where $S = |g'(r)|$ for FPI and $S= \tfrac{1}{2}$ for bisection. while $S$ is critical for linearly convergent methods, $M$ is less critical bc of the division by the square of the previous error.

##### example 11, revisited

$f'''(x) = 6x$ and at $x_c \approx 0.6823$, $M \approx 0.85$ which agrees with error ratio of displayed with iteration.

##### example 6, revisited

let $a$ be positive and consider roots of $f(x) = x^2 - a$ using newtons method.
</br></br>

\begin{align}
  x_{i+1} &= x_i - \frac{f(x_i)}{f'(x_i)} = x_i - \frac{x_i^2-a}{2x_i} \\
  &= \frac{x_i^2 + a}{2x_i} = \frac{x_i + \tfrac{a}{x_i}}{2}. \enspace\checkmark \enspace\text{babylonia!}
\end{align}
</br>

wrt convergence,
</br></br>

\begin{align}
  f'(\sqrt{a}) &= 2\sqrt{a},\enspace f''(\sqrt{a}) = 2 \\
  \\
  &\Downarrow\quad \text{quadratically convergent bc } f'(\sqrt{a}) = 2\sqrt{a} \ne 0 \\
  \\
  e_{i+} &\approx Me_i^2, \quad M = \frac{f''(r)}{2f'(r)} = \frac{2}{2\cdot 2\sqrt{a}} = \frac{1}{2\sqrt{a}}.
\end{align}

## 2 linear convergence

##### example 12

find root of $f(x) = x^2$ using newtons method.

obviously $r=0$, but
</br></br>

\begin{align}
  x_{i+1} &= x_i - \frac{f(x_i)}{f'(x_i)} \\
  &= x_i - \frac{x_i^2}{2x_i} \\
  &= \frac{x_i}{2}.
\end{align}
</br>

which gets you linear convergence of $S = \tfrac{1}{2}$. $\enspace\downarrow$ see?

###### code, example 12

In [ ]:
# example 12 # mod example 11

if __name__ == "__main__":

  import numpy as np
  import scipy as sp
  import statistics as st
  from tabulate import tabulate

  # prob-def
  f = lambda x : pow(x,2)
  df = lambda x : 2*x
  g = lambda x : x - f(x)/df(x) # newtons method

  ab = (0,2)
  h = 0.1
  h_plot = h/10

  # prob-def, generalized
  s_title = f"example 12"
  s_label = "{x^2}{2x}"
  s_label = f"$g(x) = x - \\frac{s_label}$"

  # prob-def, run-time
  x0 = st.mean(ab)
  root = (sp.optimize.root(f,[x0])).x
  ws = fpi_expanded(g,x0,worksheet=True)

  # exxing types
  we = []; e0 = 0
  for w in ws:
    ei = abs(w[1]-root)
    if e0 != 0:
      ed = ei/e0
    else:
      ed = None
    e0 = ei
    we.append([w[0],w[1],ei,ed])

  # output, table
  if True:
    s_ifmt = "03d"
    if np.max(np.absolute(ws)[:,1]) >= 10:
      s_ffmt = ".8e"
    else:
      s_ffmt = ".8f"
    print(f"\n{s_title}. x0 = {x0}.\n")
    if len(we) > 10:
      print(tabulate(we[0:6][:],headers=["i","x[i]","e[i]",'"/e[i-1]'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
      print(tabulate(we[-5:][:],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
    else:
      print(tabulate(we,headers=["i","x[i]","e[i]",'"/e[i-1]'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))



example 12. x0 = 1.

|   i |       x[i] |       e[i] |   "/e[i-1] |
|-----|------------|------------|------------|
| 000 | 1.00000000 | 1.00000000 |            |
| 001 | 0.50000000 | 0.50000000 | 0.50000000 |
| 002 | 0.25000000 | 0.25000000 | 0.50000000 |
| 003 | 0.12500000 | 0.12500000 | 0.50000000 |
| 004 | 0.06250000 | 0.06250000 | 0.50000000 |
| 005 | 0.03125000 | 0.03125000 | 0.50000000 |
|-----|------------|------------|------------|
| 022 | 0.00000024 | 0.00000024 | 0.50000000 |
| 023 | 0.00000012 | 0.00000012 | 0.50000000 |
| 024 | 0.00000006 | 0.00000006 | 0.50000000 |
| 025 | 0.00000003 | 0.00000003 | 0.50000000 |
| 026 | 0.00000001 | 0.00000001 | 0.50000000 |


##### example 13

find root of $f(x) = x^m$ using newtons method.

again, only roor $r=0$ and
</br></br>

\begin{align}
  x_{i+1} &= x_i - \frac{x_i^m}{m\, x^{m-1}} = \tfrac{m-1}{m}\,x_i \\
  \\
  &\Downarrow \quad e_i = |x_i - r| = |x_i - 0| = |x_i| \\
  \\
  e_{i+1} &= S\,e_i ,\text{ where } S = \tfrac{m-1}{m}.
\end{align}

##### theorem 12

assume $({m+1})$-times continuously differentiable function $f$ on $[a,b]$ has multiplicity $m$ root at $r$. then newtons is locally convergent to $r$ and error $e_i$ at step $i$ satisfies
</br></br>

$$\lim_{i\rightarrow\infty} \tfrac{e_{i+1}}{e_i} = S, \quad S = \tfrac{m-1}{m}.$$

##### example 14

find multiplicity of root $r = 0$ of $f(x) = sin\, x + x^2\, cos\, x - x^2 - x$ and estimate iterations required using newtons method with convergence of six decimal places beginning with $x_0 = 1$.

\begin{align}
  f(x) &= sin\,x + x^2\,cos\,x - x^2 - x \\
  f'(x) &= cos\,x + 2x\,cos\,x - x^2\,sin\,x - 2x - 1 \\
  f''(x) &= -sin\,x + 2\,cos\,x - 4x\,sin\,x - x^2\, cos\, x - 2 \\
  f'''(x) &= -cos\,x - 6\,sin\,x - 6x\,cos\,x + x^2\,sin\,x \\
  \\
  &\Downarrow \quad r = 0 \\
  \\
  f(r) &= f'(r) = f''(r) = 0,\enspace f'''(r) = -1 \quad\Rightarrow\quad m = 3 \text{ and} \\
  \\
  &\Downarrow \quad\text{by theorem 12, with linear convergence} \\
  \\
  S &= \tfrac{m-1}{m} = \tfrac{2}{3} \quad\Rightarrow\quad e_{i+1} \approx \tfrac{2}{3}\,e_i.
\end{align}

\begin{align}
  x_0 &= 1 \quad\Rightarrow\quad e_0 = 1 \\
  \\
  &\Downarrow \\
  \\
  \left( \tfrac{2}{3} \right)^n &< 0.5\times 10^{-6} \\
  \\
  &\Downarrow \\
  n &> \frac{log_{10}(0.5) - 6}{log_{10}(\tfrac{2}{3})} \approx 35.78 \quad\rightarrow\quad \text{36 iterations}.
\end{align}

###### code, example 14

In [ ]:
# example 14 # mod example 12

if __name__ == "__main__":

  import numpy as np
  import scipy as sp
  import statistics as st
  from tabulate import tabulate

  # prob-def
  f = lambda x : np.sin(x) + pow(x,2)*(np.cos(x) - 1) - x
  df = lambda x : (1+2*x)*np.cos(x) - pow(x,2)*np.sin(x) - 2*x - 1
  g = lambda x : x - f(x)/df(x) # newtons method

  ab = (0,1)
  h = 0.1
  h_plot = h/10

  # prob-def, generalized
  s_title = f"example 14"
  s_label = "{sinx + x^2cosx - x^2 -x}{cosx + 2xcosx - x^2sinx - 2x - 1}"
  s_label = f"$g(x) = x - \\frac{s_label}$"

  # prob-def, run-time
  x0 = 1
  root = 0
  ws = fpi_expanded(g,x0,tol=5e-7,worksheet=True)

  # exxing types
  we = []; e0 = 0
  for w in ws:
    ei = abs(w[1]-root)
    if e0 != 0:
      ed = ei/e0
    else:
      ed = None
    e0 = ei
    we.append([w[0],w[1],ei,ed])

  # output, table
  if True:
    s_ifmt = "03d"
    if np.max(np.absolute(ws)[:,1]) >= 10:
      s_ffmt = ".8e"
    else:
      s_ffmt = ".8f"
    print(f"\n{s_title}. x0 = {x0}.\n")
    if len(we) > 10:
      print(tabulate(we[0:6][:],headers=["i","x[i]","e[i]",'"/e[i-1]'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
      print(tabulate(we[-5:][:],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
    else:
      print(tabulate(we,headers=["i","x[i]","e[i]",'"/e[i-1]'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))



example 14. x0 = 1.

|   i |       x[i] |       e[i] |   "/e[i-1] |
|-----|------------|------------|------------|
| 000 | 1.00000000 | 1.00000000 |            |
| 001 | 0.72159024 | 0.72159024 | 0.72159024 |
| 002 | 0.52137095 | 0.52137095 | 0.72253049 |
| 003 | 0.37530831 | 0.37530831 | 0.71984890 |
| 004 | 0.26836349 | 0.26836349 | 0.71504809 |
| 005 | 0.19026161 | 0.19026161 | 0.70896981 |
|-----|------------|------------|------------|
| 031 | 0.00000622 | 0.00000622 | 0.66667262 |
| 032 | 0.00000414 | 0.00000414 | 0.66666644 |
| 033 | 0.00000276 | 0.00000276 | 0.66665531 |
| 034 | 0.00000184 | 0.00000184 | 0.66668266 |
| 035 | 0.00000123 | 0.00000123 | 0.66667984 |


##### theorem 13

if $({m+1})$-times continuously differentiable function $f$ on $[a,b]$ has multiplicity $m>1$ root at $r$, then <b>modified newtons method</b>
</br></br>

$$x_{i+1} = x_i - \frac{m\,f(x_i)}{f'(x_i)}$$
</br>

converges locally and quadratically to $r$.

ie, if multiplicity known, newtons can be improved.

##### example 14, revisited

###### code, example 14, revisited

In [ ]:
# example 14 with modified newtons

if __name__ == "__main__":

  import numpy as np
  import scipy as sp
  import statistics as st
  from tabulate import tabulate

  # prob-def
  m = 3
  f = lambda x : np.sin(x) + pow(x,2)*(np.cos(x) - 1) - x
  df = lambda x : (1+2*x)*np.cos(x) - pow(x,2)*np.sin(x) - 2*x - 1
  g = lambda x : x - m*f(x)/df(x) # newtons method

  ab = (0,1)
  h = 0.1
  h_plot = h/10

  # prob-def, generalized
  s_title = f"example 14"
  s_label = "{sinx + x^2cosx - x^2 -x}{cosx + 2xcosx - x^2sinx - 2x - 1}"
  s_label = f"$g(x) = x - \\frac{s_label}$"

  # prob-def, run-time
  x0 = 1
  root = 0
  ws = fpi_expanded(g,x0,tol=5e-7,worksheet=True) # min tol # wait for it

  # exxing types
  we = []; e0 = 0
  for w in ws:
    ei = abs(w[1]-root)
    if e0 != 0:
      ed = ei/e0
    else:
      ed = None
    e0 = ei
    we.append([w[0],w[1],ei,ed])

  # output, table
  if True:
    s_ifmt = "03d"
    if np.max(np.absolute(ws)[:,1]) >= 10:
      s_ffmt = ".8e"
    else:
      s_ffmt = ".8f"
    print(f"\n{s_title}. x0 = {x0}.\n")
    if len(we) > 10:
      print(tabulate(we[0:6][:],headers=["i","x[i]","e[i]",'"/e[i-1]'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
      print(tabulate(we[-5:][:],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
    else:
      print(tabulate(we,headers=["i","x[i]","e[i]",'"/e[i-1]'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))



example 14. x0 = 1.

|   i |       x[i] |       e[i] |   "/e[i-1] |
|-----|------------|------------|------------|
| 000 | 1.00000000 | 1.00000000 |            |
| 001 | 0.16477072 | 0.16477072 | 0.16477072 |
| 002 | 0.01620734 | 0.01620734 | 0.09836297 |
| 003 | 0.00024654 | 0.00024654 | 0.01521172 |
| 004 | 0.00000006 | 0.00000006 | 0.00024629 |


and then the conflict with machine precision wins bc its the machine. another reminder to mind the machine.
</br></br>

ie, backwards error is driven near $\epsilon_{\text{mach}}$ but forward error $x_i$ is several orders of magnitude larger.

## 3 more fail

##### example 15

apply newtons method to $f(x) = 4x^4 - 6x^2 - \tfrac{11}{4}$ with $x_0 = \tfrac{1}{2}$.

the function has roots bc it is continuous and negative at $x=0$ and goes to $+\infty$ for large positive and negative $x$.

$$x_{i+1} = x_i - \frac{4x_i^4 - 6x_i^2 - \tfrac{11}{4}}{16x_i^3 - 12x_i}.$$

however, $x_1 = -\tfrac{1}{2} \enspace\mapsto\enspace x_2 = -\tfrac{1}{2} \enspace\mapsto\enspace$ lol.

###### code, example 15

thats worth some code.

In [ ]:
# example 15 # repurposed example 11

if __name__ == "__main__":

  import numpy as np
  import scipy as sp
  from tabulate import tabulate

  # prob-def
  f = lambda x : 4*pow(x,4) - 6*pow(x,2) - 11/4
  df = lambda x : 16*pow(x,3) - 12*x
  g = lambda x : x - f(x)/df(x) # newtons method

  ab = (-1.31,1.31)
  h = 0.1
  h_plot = h/10

  # prob-def, generalized
  s_title = f"example 15 ~ newton fails"
  s_label = "{4x^4 - 6x^2 - \tfrac{11}{4}}{16x^3 - 12x}"
  s_label = f"$g(x) = x - \\frac{s_label}$"

  # prob-def, run-time
  x0 = 0.5
  root = (sp.optimize.root(f,x0=[ab[1]])).x
  roots = np.array([-root,root])
  ws = fpi_expanded(g,x0,max_iter=5,worksheet=True)

  # cfg, runtime
  iterations = 3

  # exxing types
  we = []; e0 = 0
  for w in ws:
    ei = abs(w[1]-root)
    if e0 != 0:
      ed = ei/pow(e0,2)
    else:
      ed = None
    e0 = ei
    we.append([w[0],w[1],ei,ed])

  # output, table
  if True:
    s_ifmt = "03d"
    if np.max(np.absolute(ws)[:,1]) >= 10:
      s_ffmt = ".8e"
    else:
      s_ffmt = ".8f"
    print(f"\n{s_title}. x0 = {x0}.\n")
    if len(we) > 10:
      print(tabulate(we[0:6][:],headers=["i","x[i]","e[i]",'"/e[i-1]^2'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
      print(tabulate(we[-5:][:],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))
    else:
      print(tabulate(we,headers=["i","x[i]","e[i]",'"/e[i-1]^2'],intfmt=s_ifmt,floatfmt=s_ffmt,tablefmt="github"))

  # output, cobweb plot
  if False:
    import matplotlib.pyplot as plt

    # g(x)
    s_grey = "0.5"
    plt.plot([ab[0]-h,ab[1]+h],[ab[0]-h,ab[1]+h],ls=":",c=s_grey)
    xs_plot = np.arange(ab[0]-h,ab[1]+h_plot/2,h_plot)
    gs_plot = g(xs_plot)
    plt.plot(xs_plot,gs_plot,label=s_label,c=s_grey)

    # fpi
    s_blue = "C00"
    xs = []; ys = []
    imax = min(len(ws),5)
    for i in range(1,imax):
      xs.append(ws[i-1][1])
      ys.append(ws[i][1])
    plt.scatter(xs,ys,c=s_blue,marker="*")

    # fpi vs x=y diagonal
    y0 =0
    for x,y in zip(xs,ys):
      if y0 == 0:
        plt.plot([x,x],[y0,y],label="fpi",c=s_blue,alpha=0.5)
      else:
        plt.plot([x,x],[y0,y],c=s_blue,alpha=0.5)
      plt.arrow(x,y,y-x,0,color=s_blue,alpha=0.5,length_includes_head=True,head_width=0.025)
      y0 = y

    plt.title(f"{s_title}, {s_label}")
    plt.grid()
    plt.legend()
    plt.show()

  # output, animiation # True requires "ani" in a separate code cell
  if True:
    from itertools import count
    import math as m
    import matplotlib.animation
    import matplotlib.pyplot as plt
    #import numpy as np # already

    # data, true, scatter
    h = 0.1
    xs = np.arange(ab[0],ab[1]+h,h)
    ys = f(xs)

    # data, true, plot
    h_plot = h/10
    xs_plot_a = ab[0]-h
    xs_plot_b = ab[1]+h
    xs_plot = np.arange(xs_plot_a,xs_plot_b+h_plot,h_plot)
    ys_plot = f(xs_plot)

    # plot, animation
    plt.rcParams["animation.html"] = "jshtml"
    plt.rcParams['figure.dpi'] = 100
    plt.ioff()
    fig,ax = plt.subplots()
    index = count() # used in animate()

    # plot, actual
    ax.plot(xs_plot,ys_plot,zorder=1)
    ymin = m.floor(ys_plot.min()/h)*h - h
    ymax = m.ceil(ys_plot.max()/h)*h + h
    ax.set_xlim(xs_plot_a,xs_plot_b)
    ax.set_ylim(ymin,ymax)
    ax2 = ax.twinx()
    ax.set_title(s_title)
    ax.grid()

    # plot, iterations
    zorder = 10
    ws = np.array(ws) # for referential
    def animate(t):
      ax2.cla()
      k = next(index)
      if k == 2:
        ax.scatter(roots,f(roots),color="C01",marker="*",s=200,zorder=2)
      if k > 2:
        step = 3
        imax = k - 2 # first frame on ax2
        for iset in range(0,imax,step):
          i = iset // step
          color = f"C{i+2:02d}" # C02,C03,C04,...

          jmax = min(imax-iset,step)
          for j in range(jmax):
            if j > 1:
              x = ws[i,1]
              x1 = ws[i+1,1]
              y = f(x)
              ya = y*((xs_plot_a-x)/(x-x1) + 1)
              yb = y*((xs_plot_b-x)/(x-x1) + 1)
              ax2.plot([xs_plot_a,xs_plot_b],[ya,yb],label="$m_{i}$",color=color)
            else:
              if j == 0:
                x = ws[i,1]
                xs_j = [x]
                ys_j = [0]
                acbs = ["x"]
              elif j == 1:
                x = ws[i,1]
                xs_j = [x]
                ys_j = [f(x)]
                acbs = ["y"]
              ax2.scatter(xs_j,ys_j,color=color,zorder=zorder+i)
              for acb,x,y in zip(acbs,xs_j,ys_j):
                label = "$" + acb+ "_{" + str(i) + "}$"
                ax2.text(x-h_plot*2,y-h*5/2,label,color=color,fontweight="bold",zorder=zorder*2)

      # inside ani function!!
      ax2.set_xlim(xs_plot_a,xs_plot_b)
      ax2.set_ylim(ymin,ymax)

    ani = matplotlib.animation.FuncAnimation(fig,animate,frames=11,interval=11)

    # show static
    if False:
      plt.show()



example 15 ~ newton fails. x0 = 0.5.

|   i |        x[i] |       e[i] |   "/e[i-1]^2 |
|-----|-------------|------------|--------------|
| 000 |  0.50000000 | 0.86676040 |              |
| 001 | -0.50000000 | 1.86676040 |   2.48479439 |
| 002 |  0.50000000 | 0.86676040 |   0.24872641 |
| 003 | -0.50000000 | 1.86676040 |   2.48479439 |
| 004 |  0.50000000 | 0.86676040 |   0.24872641 |
| 005 | -0.50000000 | 1.86676040 |   2.48479439 |


In [ ]:
ani

in addition to examples 14 and 15, if $f'(x_i) = 0$. also, iterations unto infinity or mimicry of an rng. however, theorems 11 and 12 guarantee a neighborhood of initial guesses surrounding each root for which convergence to that root is assured.

</br></br></br></br>

## resources

###### code, fpi, from lecture 01_02

In [1]:
# algorithm, expanded for lecture 01_02

def fpi_expanded(g,x,tol=1e-8,max_iter=100,worksheet=False):

  count = 0
  if worksheet:
    data = []

  gx = g(x)
  if worksheet:
    data.append([count,x])
  while (abs(gx-x) > tol) and (count < max_iter):
    x = gx
    gx = g(x)

    count += 1
    if worksheet:
      data.append([count,x])
  if worksheet:
    return data
  else:
    return x


</br></br></br></br></br></br></br></br></br></br>